In [1]:
# Import relevant functionality
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults # search tool
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver # checkpointer
from langgraph.prebuilt import create_react_agent # tool?
import os

with open('../key.txt', "r", encoding="utf-8") as file:
    content = file.readline()
    
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_API_KEY'] = content.split(' ')[-1]
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_PROJECT'] = "project-test-1"


# Create the agent
memory = MemorySaver()
model = ChatOpenAI(model_name="gpt-4o-mini")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory) # model + tool + checkpoint

config = {"configurable": {"thread_id": "abc123"}}

# Use the agent & debug steps
if 0:
    for chunk in agent_executor.invoke( # no tool 
        {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
    ):
        print(chunk)
        print("----")
    
    for chunk in agent_executor.stream( # call tool
        {"messages": [HumanMessage(content="How many people are there in the place live?")]}, config
    ):
        print(chunk)
        print("----")

### Step-by-step: Agent

In [ ]:
## Without agent, it doesn't act
if 0:
    model_with_tools = model.bind_tools(tools)
    response = model_with_tools.invoke([HumanMessage(content="What's the weather in Wuhan today?")])
    print(response.tool_calls)

## With agent, the steps. How does model know how many steps?
if 1: # call_model -> (re-direct)tool -> (searched-2-results)AI -> End 
    agent_executor = create_react_agent(model, tools, checkpointer=memory)
    response = agent_executor.invoke(
        {"messages": [HumanMessage(content="whats the weather in Wuhan today?")]}, config
    )
    print(response["messages"])

## React Prompt

In [9]:
from langchain import hub

prompt = hub.pull("hwchase17/react") # get from hub


In [ ]:
model_with_tools